In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
amazon_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
amazon_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [4]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_table = amazon_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

In [5]:
# Load in a swl function to use columns
from pyspark.sql.functions import col

cleaned_df = vine_table.filter(col("total_votes") >= "20")
cleaned_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QXNQBTH7LIUB|          3|            9|         22|   N|                Y|
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|     

In [6]:
# Filter and retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
#from pyspark.sql.types import DoubleType
# Dividing Columns - https://stackoverflow.com/questions/59351771/how-to-divide-each-column-in-a-df-by-other-columns-in-pyspark

helpful_vote_df = cleaned_df.withColumn('Helpful Vote %', cleaned_df.helpful_votes/cleaned_df.total_votes)
helpful_vote_df.show()

helpful_vote_df_cleaned = helpful_vote_df.filter(col("Helpful Vote %") >= "0.5")
helpful_vote_df_cleaned.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    Helpful Vote %|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1QXNQBTH7LIUB|          3|            9|         22|   N|                Y|0.4090909090909091|
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|0.9325842696629213|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|0.8181818181818182|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|               1.0|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|0.8666666666666667|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|0.7428571428571429|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|0.8823529411764706|
|R3RNY5OLY451GS|          1|  

In [7]:
# Filter df to find that retrieves rows with over 50% helpful votes and was written as part of the Vine program

vine_review_df = helpful_vote_df_cleaned.filter(col("vine") == "Y")
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    Helpful Vote %|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R20O97P5V9CZ21|          3|           21|         27|   Y|                N|0.7777777777777778|
| RCNB7YGLWKCV5|          5|           23|         30|   Y|                N|0.7666666666666667|
| RX1ZUVGBH3HKC|          5|          814|        848|   Y|                N|0.9599056603773585|
| RRS5AGL10DYU3|          3|           25|         29|   Y|                N|0.8620689655172413|
| R5YFC5G8V2B3I|          5|          104|        117|   Y|                N|0.8888888888888888|
|R1IDQ0G1S7J9TN|          5|          178|        196|   Y|                N|0.9081632653061225|
| RKXNZX22HQZEE|          3|          182|        192|   Y|                N|0.9479166666666666|
|R2K45IY0NO70ZI|          5|  

In [8]:
# Filter df to find that retrieves rows with over 50% helpful votes and was not written as part of the Vine program

non_vine_review_df = helpful_vote_df_cleaned.filter(col("vine") == "N")
non_vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    Helpful Vote %|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|0.9325842696629213|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|0.8181818181818182|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|               1.0|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|0.8666666666666667|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|0.7428571428571429|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|0.8823529411764706|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|0.8333333333333334|
|R16LPVYSJOAH1T|          1|  

In [9]:
# Determine the total number of reviews
total_reviews = amazon_df.count()
total_reviews

96901

In [32]:
# Determine the total number of paid reviews
total_paid_reviews = amazon_df.filter(col("vine") == "Y").count()
total_paid_reviews

248

In [31]:
# Determine the total number of unpaid reviews
total_unpaid_reviews = amazon_df.filter(col("vine") == "N").count()
total_unpaid_reviews

96653

In [10]:
# Determine the total number of 5-star reviews
five_star_reviews = amazon_df.filter(col("star_rating") == "5").count()
five_star_reviews

49704

In [18]:
# Determine the total number of paid 5-star reviews
# https://www.geeksforgeeks.org/pyspark-filter-dataframe-based-on-multiple-conditions/
paid_five_star_reviews = amazon_df.filter((col("vine") == "Y") &
                                          (col("star_rating") == "5")).count()
paid_five_star_reviews


112

In [28]:
# Determine precentage of paid 5-star reviews among total paid reviews 
paid_five_star_reviews_percent = (paid_five_star_reviews / total_paid_reviews) * 100

paid_five_star_reviews_percent

45.16129032258064

In [19]:
# Determine the total number of unpaiid 5-star reviews
unpaid_five_star_reviews = amazon_df.filter((col("vine") == "N") &
                                          (col("star_rating") == "5")).count()
unpaid_five_star_reviews

49592

In [30]:
# Determine precentage of unpaid 5-star reviews among total unpaid reviews 
unpaid_five_star_reviews_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100

unpaid_five_star_reviews_percent

51.30932304222321

In [36]:
vine_review_df.describe().show()

+-------+--------------+------------------+------------------+------------------+----+-----------------+-------------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|     Helpful Vote %|
+-------+--------------+------------------+------------------+------------------+----+-----------------+-------------------+
|  count|            35|                35|                35|                35|  35|               35|                 35|
|   mean|          null| 4.371428571428571| 75.28571428571429| 80.57142857142857|null|             null| 0.9057453055399282|
| stddev|          null|0.7702449681266148|136.07082313105562|141.57232317467856|null|             null|0.06298380544915456|
|    min|R15NQCQUNOSI4U|                 2|                15|                20|   Y|                N|               0.75|
|    max| RX7QEPPDUZLO7|                 5|               814|               848|   Y|                Y| 0.9807692307692307|


In [37]:
non_vine_review_df.describe().show()

+-------+--------------+------------------+-----------------+-----------------+----+-----------------+-------------------+
|summary|     review_id|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|     Helpful Vote %|
+-------+--------------+------------------+-----------------+-----------------+----+-----------------+-------------------+
|  count|          4957|              4957|             4957|             4957|4957|             4957|               4957|
|   mean|          null| 3.246520072624571|48.58644341335485|53.22796045995562|null|             null| 0.8886292696877135|
| stddev|          null|1.7280125973419265|77.04607780493804|79.32273398498282|null|             null|0.10134464144927993|
|    min|R100212BMGLGI0|                 1|               10|               20|   N|                N|                0.5|
|    max| RZZQQEWCN58IM|                 5|             1724|             1761|   N|                Y|                1.0|
+-------+-------